# JavaScript Macrotask Queue – Deep Dive for Students

## 1. What is a Macrotask?
A **macrotask** (aka *task*) is a unit of work the event loop executes from the **macrotask queue**. After a macrotask finishes, the browser runs **all queued microtasks** before picking the next macrotask.

**Common characteristics:**
- Run one-at-a-time.
- May be delayed or throttled by the browser.
- Originates from many different APIs.

---

## 2. Common Macrotask Sources in Browsers

### a) Timers
- `setTimeout(fn, ms)`
- `setInterval(fn, ms)`
- Clamping rules: nested timers clamp to ≥4ms, background tabs may be throttled heavily.

```js
setTimeout(() => console.log("timeout macro"), 0);
setInterval(() => console.log("interval macro"), 1000);
```

### b) DOM/UI Events
- Examples: `click`, `scroll`, `keydown`, `input`, `submit`.
- Each event dispatch is a macrotask; microtasks in the handler run before the browser processes the next event.

```js
document.addEventListener("click", () => {
  console.log("click macro");
  Promise.resolve().then(() => console.log("micro after click"));
});
```

### c) `MessageChannel` / `postMessage`
- Faster than `setTimeout(…, 0)`.
- No clamping like timers.

```js
const ch = new MessageChannel();
ch.port1.onmessage = () => console.log("messagechannel macro");
ch.port2.postMessage(null);
```

### d) Networking & I/O
- XHR: `readystatechange`, `load`, `error`
- WebSocket: `message`, `open`, `close`, `error`
- EventSource: `message`
- FileReader: `load`, `error`, `progress`
- IndexedDB: `success`, `error`, `complete`
- Geolocation: callbacks

```js
const ws = new WebSocket("wss://example.com");
ws.addEventListener("message", e => {
  console.log("websocket message macro");
  Promise.resolve().then(() => console.log("micro after ws"));
});
```

### e) History & Navigation Events
- `popstate`, `hashchange`
- `pageshow`, `pagehide`, `visibilitychange`

```js
window.addEventListener("popstate", () => console.log("popstate macro"));
```

### f) Storage / Cross-document Events
- `storage` event fires in *other* tabs when `localStorage` changes.

```js
window.addEventListener("storage", e => console.log("storage macro", e.key));
```

### g) Workers Messaging
- `worker.postMessage` → main thread gets a macrotask, same the other way.

```js
const w = new Worker("worker.js");
w.onmessage = () => console.log("worker→main macro");
w.postMessage("hi");
```

---

## 3. What Are NOT Macrotasks?
- **Microtasks**: `Promise.then/catch/finally`, `queueMicrotask`, `MutationObserver`.
- **requestAnimationFrame (rAF)**: runs before the next paint.
- **requestIdleCallback**: runs during idle periods.
- **ResizeObserver / IntersectionObserver**: special rendering-phase callbacks.

---

## 4. Cheatsheet
- **ASAP after current code** → microtask.
- **Yield to UI / avoid starving loop** → macrotask.
- **Sync with frames** → `requestAnimationFrame`.
- **Low priority work** → `requestIdleCallback`.

---

## 5. Mini Demo Comparing Sources
```html
<script>
console.log("script start");

setTimeout(() => console.log("timeout (macro)"), 0);

const ch = new MessageChannel();
ch.port1.onmessage = () => console.log("messagechannel (macro)");
ch.port2.postMessage(null);

window.postMessage("x", "*");
window.addEventListener("message", () => console.log("window.postMessage (macro)"), { once: true });

Promise.resolve().then(() => console.log("promise (micro)"));

console.log("script end");
</script>
```
**Expected order:**
```
script start
script end
promise (micro)
messagechannel (macro) / window.postMessage (macro)
timeout (macro)
```

---
